<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex11_Optimal_Active_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investments: Theory and Data Analysis**, Bates, Boyer, and Fletcher


# Example Chapter 11: Optimal Active Portfolio
In this example we estimate the optimal active tilt towards the zero-cost portfolio that goes long decile 1 short-term reversal portfolio and short decile 10 short-term reversal portfolio from the Ken French Data Library.


### Imports and Setup

In [ ]:
# Import packages
%pip install -q simple-finance
import simple_finance as sf
import requests
import pandas as pd
import numpy as np

# Establish display options when printing DataFrames
pd.set_option('display.max_columns', None)   # Show all columns without truncation
pd.set_option('display.width', 1000)   # Set the display width so output stays on one line
pd.set_option("display.max_rows", 20) # Force truncation if DataFrame has more than 20 rows

### Load in Data Using the Python Function
See Ex07-Ken_French.ipynb for a complete description of the function in the course GiHub repo.


In [ ]:
# Get returns on short-term reveral deciles
df=sf.get_ff_strategies('shorttermreversal', details=False)
print(df)

### Define Inputs
In the code cell below we define the covariance matrix and the vector of expected returns.  Here you can enter your own covriance matrix and vector of expected returns as desired, with any number of securities. You can also load these inputs from your account on Google Drive, or your own GitHub repo.   

In [ ]:
df = df.copy()  # avoid SettingWithCopyWarning

# Define the zero-cost portfolio
df['Dec1_Dec10'] = df['Dec 1'] - df['Dec 10']

# Estimate Expected excess returns
expected_returns = df[['mkt-rf', 'Dec1_Dec10']].mean().to_numpy()

# Estimate the covariance matrix
covariance_matrix = df[['mkt-rf', 'Dec1_Dec10']].cov().to_numpy()
print(expected_returns)
print(covariance_matrix)

### Define the Tangent Portfolio
Function: `tangent_portfolio`   

**Inputs**
* `expected_returns`: the array of expected returns
* `covariance_matrix`: the covariance matrix
* `factors`: if `True` assumes $E[r]$ and $\Sigma$ are computed from zero-cost factor inputs (excess returns instead of total returns).

**Output**
* `tangent_weights`: the weights of the tangent portfolio
* `tangent_return`: the expected return of the tangent portfolio
* `tangent_volatility`: the volatility of the tangent portfolio


In [ ]:
tangent_weights, tangent_return, tangent_volatility = sf.tangent_portfolio(expected_returns, covariance_matrix, factors=True)

print(tangent_weights)
print(tangent_return)
print(tangent_volatility)

### Test Trading Strategy

In [ ]:
# For comparison with Ex07-Ken_French.ipynb
#tangent_weights = np.array([1, 0.250])

# Create realized excess portfolio returns
excess = df[['mkt-rf', 'Dec1_Dec10']].to_numpy()   # shape (T,2)

# Calculate the excess returns as a 1D array
# @ performs an inner product
excess_array = excess @ tangent_weights

# Convert the Series to a DataFrame
excess = pd.DataFrame(excess_array, index=df.index, columns=['excess'])

# compute total return
total_return = excess['excess'] + df['rf']

# Get monthly mean returns, volatility, and sharpe
# Call .mean() to get a scalar mean from the total_return Series
mean_monthly = total_return.mean()
# Select the 'excess' column before calling .mean() as 'excess' is a DataFrame
mean_excess_monthly  = excess['excess'].mean()
# Select the 'excess' column before calling .std() as 'excess' is a DataFrame
std_monthly  = total_return.std()
sharpe_monthly = mean_excess_monthly / std_monthly

# annualize
ann_return = 12 * mean_monthly
ann_vol    = np.sqrt(12) * std_monthly
ann_sharpe = sharpe_monthly * np.sqrt(12)

# Print results
print(f"Average Return: {ann_return:.3f}")
print(f"Annualized Volatility: {ann_vol:.3f}")
print(f"Annualized Sharpe Ratio: {ann_sharpe:.3f}")